# TUTORIAL IMAGE PROCESSING FOR MATERIAL SCIENCE (I)

## Import the libraries 
activating the libraries to be used in the following

In [ ]:
from PIL import Image      # Modulo basico de cargar imagenes
import numpy as np         # Modulo arrays
from pathlib import Path

# import matplotlib.pyplot as plt  # You need matplotlib to visualize if not napari

import pandas as pd
import numpy as np
import skimage
import skimage.measure
from os import listdir

# For future tuto
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import DBSCAN

# This is for napari in jupyter env.
%gui qt5 
import napari

## 0.1 LOADING 2D IMAGES
It is more handy to convert the grayscale images to a binary one in Imagej/Fiji.

In [3]:
## Declaring the paths to the folders of work
folder_to_read = Path(r"D:\OneDrive - Universidad Politécnica de Madrid\2_Tareas\20201007_Iker_cluster") 
folder_to_save = Path(r"D:\OneDrive - Universidad Politécnica de Madrid\2_Tareas\20201007_Iker_cluster")

## Name of the files
name1 = 'Classification result-1.tif'
# name2 = 'Classification result-2.tif'
# name3 = 'Classification result-3.tif'
# name4 = 'Classification result-4.tif'

ruta_data1 = folder_to_read / name1
# ruta_data2 = folder_to_read / name2
# ruta_data3 = folder_to_read / name3
# ruta_data4 = folder_to_read / name4

## Assigning images to variables
# Check the different steps
'''
First we are using  objects:ruta_data1,ruta_data2.. of type Path (realised of the upcase 'P') to point to the path of the file in our computer. This would make it work for any Operative System (linux, windows, mac)
Second the Image class (again realised of the 'I') from the PIL library enables to read the image in the format we specify.
Third, and last the objects created by the class Image are turned into another object of other type> numpy array. These objects behave as what we would expect of a math matrix.
'''
img1 = np.array(Image.open(str(ruta_data1), mode='r'))
img2 = np.array(Image.open(str(ruta_data2), mode='r'))
img3 = np.array(Image.open(str(ruta_data3), mode='r'))
img4 = np.array(Image.open(str(ruta_data4), mode='r'))

img1.shape,img2.shape,img3.shape,img4.shape
#carga direcciones al revés

((1755, 1883), (1755, 1883), (1755, 1883), (1755, 1883))

### Inspection of the images visually

In [18]:
viewer = napari.view_image(img1)

## 0.1.1 LOADING 3D IMAGES

### This is the function to read tiff images. Mind the 'folder' parameter to set whether it is a multiframe tiff or multiple tiff images.

In [5]:
def napari_read_tiff(pathlibpath,start=0, folder=False, nframes='all'):
    '''
    Read a multiframe tif image and loads it in a numpy array
    '''
    if not folder:
        with Image.open(str(pathlibpath), mode='r') as img:
            stack = []
            if nframes == 'all':

                try:
                    for i in range(start,img.n_frames):
                        img.seek(i)
                        stack.append(np.array(img))
                except:
                    print(i)
                    pass
        #     else:
        #         for i in range(start,nframes):
        #             img.seek(i)
        #             stack.append(np.array(img))
    if folder:
        files = listdir(pathlibpath)
        stack = []
        for file in files:
            filepath = pathlibpath / file
            with Image.open(str(filepath),mode='r') as img:
                stack.append(np.array(img))
    
    return np.transpose(np.array(stack), (1, 2, 0))

### Napari has different coordinate system than ImageJ. I use this func to make it equivalent to imagej

In [6]:
def axis_napari(array):
    array = np.transpose(array, (2,0,1))
    return(array)

In [19]:
## Multiframe tiff
path_to_img = Path(r"D:\OneDrive - Universidad Politécnica de Madrid\2_Tareas\20201007_Iker_cluster\3D_test\multiframe_3d.tif")
multiframe = napari_read_tiff(path_to_img,folder=False)
napari.view_image(axis_napari(multiframe))

## Multiple tiff images
path_to_folder = Path(r"D:\OneDrive - Universidad Politécnica de Madrid\2_Tareas\20201007_Iker_cluster\3D_test\Multiple_tif_imgs")
multiimgs = napari_read_tiff(path_to_folder,folder=True)
# napari.view_image(axis_napari(multiimgs))

### Following sections are only performed in 2D images for simplicity sake.

## 2. DETECT THE INDEPENDENT REGIONS OF INTEREST
Bear in mind the types of connectivity. Check the docs

In [7]:
labels1 = skimage.measure.label(img1, background=0, connectivity=2)

In [20]:
# the viewer ran before must be open.
viewer.add_labels(labels1)

<Labels layer 'labels1' at 0x1e11cd23588>

## 3. COMPUTE PROPERTIES FOR EACH REGION

### Option 1: Regionprops_table

In [21]:
props_table = skimage.measure.regionprops_table(labels1, properties=('label',
                                                                     'centroid',
                                                                     'area',
                                                                     'eccentricity',
                                                                     'equivalent_diameter',
                                                                     'major_axis_length',
                                                                     'minor_axis_length'))
props_table

{'label': array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
         14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
         27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
         40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
         53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
         66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
         79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
         92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104,
        105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117,
        118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130,
        131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143,
        144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156,
        157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169,
        170, 171, 172, 173, 174, 175, 176,

In [10]:
## Better the table in a pandas dataframe format
df_props = pd.DataFrame(props_table)
df_props

,label,centroid-0,centroid-1,area,eccentricity,equivalent_diameter,major_axis_length,minor_axis_length
0,1,94,1647,3,0.816497,1.954410,2.309401,1.333333
1,2,112,1069,27,0.990165,5.863230,17.711145,2.477855
2,3,112,1740,7,0.984986,2.985411,7.802529,1.347002
3,4,111,1737,3,0.973723,1.954410,3.677089,0.837402
4,5,119,1734,6,0.899735,2.763953,4.320494,1.885618
...,...,...,...,...,...,...,...,...
468,469,1753,136,6,0.960659,2.763953,4.849661,1.346893
469,470,1753,30,2,1.000000,1.595769,2.000000,0.000000
470,471,1753,1583,3,0.816497,1.954410,2.309401,1.333333
471,472,1753,1822,2,1.000000,1.595769,2.000000,0.000000


### Visualizing centroids

In [28]:
viewer.add_points(df_props[['centroid-0','centroid-1']], size=5)

<Points layer 'Points' at 0x1e11cad6588>

### Option 2: Regionprops

In [23]:
l_regions = skimage.measure.regionprops(labels1)

In [28]:
## Example
# Creation of an empty list then append the values

l_label = []
l_centroid = []
for item in l_regions:
    l_label.append(item.label)
    l_centroid.append(item.centroid)

    
# as a numpy array 1D, kind of  "math vector"
a_area = np.array(l_area)

# Proving it is the same as option 1 but in another format
l_label[:5], l_centroid[:5]

([1, 2, 3, 4, 5],
 [(94.66666666666667, 1647.6666666666667),
  (112.18518518518519, 1069.2962962962963),
  (112.42857142857143, 1740.4285714285713),
  (111.66666666666667, 1737.0),
  (119.5, 1734.8333333333333)])

## exporting the pandas dataframe as csv file

In [31]:
df_props.to_csv(folder_to_save / 'region_props.csv')

In [32]:
str(folder_to_save / 'region_props.csv')

'D:\\OneDrive - Universidad Politécnica de Madrid\\2_Tareas\\20201007_Iker_cluster\\imagetutorial.csv'